# Data Download

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/kermany2018")

print("Path to dataset files:", path)

100%|██████████| 10.8G/10.8G [05:19<00:00, 36.5MB/s]  

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/paultimothymooney/kermany2018/versions/2


In [ ]:
import os

os.makedirs("./data/", exist_ok=True)

for cohort in ["CNV", "NORMAL"]:

    ! cp -r "{path}/OCT2017 /train/{cohort}/" ./data/

# Train the Model

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, StableDiffusionPipeline
from transformers import CLIPTextModel, CLIPTokenizer
from accelerate import Accelerator

In [3]:
class ImgDataset(Dataset):

    def __init__(self, root_dir:str, tokenizer, cohort:str = "CNV", resolution=512, max_length=77, dataset_size:int = 6400) -> None:

        self.files = [
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

        if len(self.files) > dataset_size: self.files = self.files[:dataset_size]

        self.tokenizer = tokenizer

        self.transform = transforms.Compose([
            transforms.Resize((resolution, resolution), transforms.InterpolationMode.LANCZOS),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ])

        self.prompt = f"OCT scan showing {cohort}"
        self.max_length = max_length

    def __len__(self) -> int:

        return len(self.files)

    def __getitem__(self, idx:int) -> dict:

        img = Image.open(self.files[idx]).convert("RGB")
        img = self.transform(img)

        tokens = self.tokenizer(
            self.prompt,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        return {"pixel_values": img, "input_ids": tokens.input_ids.squeeze(0)}

In [5]:
def train(
    pretrained_model: str,
    data_dir: str,
    output_dir: str = "sd_cnv_finetuned",
    cohort: str = "CNV",
    resolution: int = 512,
    dataset_size:int = 6400,
    batch_size: int = 4,
    learning_rate: float = 1e-4,
    epochs: int = 5,
    grad_accum_steps: int = 1,
    save_steps: int = 1000,
    resume_checkpoint: str = None,
    unet: UNet2DConditionModel = None,
    accelerator: Accelerator = None,
):

    # 1) Prepare output directory, accelerator & device
    os.makedirs(output_dir, exist_ok=True)
    accel = accelerator or Accelerator()
    device = accel.device

    # 2) Load or resume UNet
    if resume_checkpoint and unet is None:
        unet = UNet2DConditionModel.from_pretrained(resume_checkpoint).to(device)
    elif unet is None:
        unet = UNet2DConditionModel.from_pretrained(pretrained_model, subfolder="unet").to(device)

    # 3) Load & freeze tokenizer + text encoder
    tokenizer    = CLIPTokenizer.from_pretrained(pretrained_model, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(pretrained_model, subfolder="text_encoder").to(device)
    text_encoder.requires_grad_(False)

    # 4) Load & freeze VAE
    vae = AutoencoderKL.from_pretrained(pretrained_model, subfolder="vae").to(device)
    vae.requires_grad_(False)

    # 5) Load noise scheduler
    scheduler = DDPMScheduler.from_pretrained(pretrained_model, subfolder="scheduler")

    # 6) Prepare dataset & dataloader (assumes CNVDataset is defined)
    dataset    = ImgDataset(data_dir, tokenizer, cohort = cohort, resolution=resolution, dataset_size = dataset_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 7) Optimizer (only UNet params)
    optimizer = torch.optim.AdamW(unet.parameters(), lr=learning_rate)

    # 8) Wrap models, optimizer, and dataloader for mixed‑precision / distributed
    unet, optimizer, dataloader = accel.prepare(unet, optimizer, dataloader)

    # 9) Resume state if requested
    if resume_checkpoint:
        accel.load_state(resume_checkpoint)
        # pick up where you left off
        global_step = int(resume_checkpoint.rsplit("_", 1)[-1])
    else:
        global_step = 0

    # 10) Training loop
    for epoch in range(1, epochs + 1):
        unet.train()
        for batch in dataloader:
            with accel.accumulate(unet):
                # Encode images to latents
                pixels  = batch["pixel_values"].to(device)
                latents = vae.encode(pixels).latent_dist.sample() * 0.18215

                # Add noise
                noise     = torch.randn_like(latents)
                timesteps = torch.randint(0, scheduler.num_train_timesteps,
                                          (latents.shape[0],), device=device)
                noisy_latents = scheduler.add_noise(latents, noise, timesteps)

                # Text conditioning
                input_ids             = batch["input_ids"].to(device)
                encoder_hidden_states = text_encoder(input_ids)[0]

                # Noise prediction & loss
                pred_noise = unet(noisy_latents, timesteps, encoder_hidden_states).sample
                loss       = torch.nn.functional.mse_loss(pred_noise, noise)

                # Backpropagate
                accel.backward(loss)
                optimizer.step()
                optimizer.zero_grad()

            global_step += 1
            if global_step % save_steps == 0:
                accel.wait_for_everyone()
                ckpt_dir = os.path.join(output_dir, f"checkpoint_{global_step}")
                unet.save_pretrained(ckpt_dir)
                if accel.is_main_process:
                    tokenizer.save_pretrained(ckpt_dir)
                accel.save_state(ckpt_dir)

        print(f"Epoch {epoch}/{epochs} complete")

    # 11) Final save
    accel.wait_for_everyone()
    final_dir = os.path.join(output_dir, "final_unet")
    unet.save_pretrained(final_dir)
    if accel.is_main_process:
        tokenizer.save_pretrained(output_dir)
    print("Fine-tuning complete — models saved to", output_dir)

In [ ]:
from accelerate import Accelerator
# Optionally load the checkpoint’s state:
# from https://huggingface.co/nota-ai/bk-sdm-small
accel = Accelerator()
train(
    pretrained_model="nota-ai/bk-sdm-small",
    data_dir="./data/CNV",
    output_dir="./models/sd_cnv_finetuned",
    cohort = "CNV",
    resolution=512,
    batch_size=4,
    learning_rate=1e-4,
    epochs=5,
    grad_accum_steps=1,
    save_steps=1000,
    resume_checkpoint=None,
    accelerator=accel
)

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

/home/projects/Novo-Nordisk_Anomaly-Detection/.venv/lib/python3.13/site-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


In [ ]:
# if you get out of memory issues, run this cell

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# for me to download the files from colab

!zip -r /content/checkpoints_11000.zip /content/sd_cnv_finetuned/checkpoint_11000/

  adding: content/sd_cnv_finetuned/checkpoint_11000/ (stored 0%)
  adding: content/sd_cnv_finetuned/checkpoint_11000/tokenizer_config.json (deflated 63%)
  adding: content/sd_cnv_finetuned/checkpoint_11000/random_states_0.pkl (deflated 25%)
  adding: content/sd_cnv_finetuned/checkpoint_11000/model.safetensors

# Inference

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTokenizer

device = "cuda"  # or "cpu" if no GPU

# 1) Load the base pipeline (with original VAE, text-encoder, scheduler, tokenizer)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)


# 2) Load your fine-tuned UNet
finetuned_unet = UNet2DConditionModel.from_pretrained(
    "/content/sd_cnv_finetuned/checkpoint_13000",
    torch_dtype=torch.float16
).to(device)

# 3) Replace the pipeline’s UNet
pipe.unet = finetuned_unet

# 4) (Optional) Load & swap in your checkpoint’s tokenizer
ckpt_tokenizer = CLIPTokenizer.from_pretrained(
    "/content/sd_cnv_finetuned/checkpoint_5000"
)
pipe.tokenizer = ckpt_tokenizer

# 5) Generate!
prompt = "OCT scan showing CNV"
out = pipe(
    prompt,
    num_inference_steps=50,
    guidance_scale=7.5
)
img = out.images[0]
img.save("cnv_finetuned_example.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]